In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import os
import sys

import numpy as np
import requests
from google.cloud import bigquery
from loguru import logger

module_path = os.path.abspath("..")
sys.path.append(module_path)

from src.utils.notebooks import patch_kfp

patch_kfp()

from src.components.data import get_data_version

# Remove default logger and set level to DEBUG
logger.remove()
logger.add(sys.stderr, level="DEBUG")

In [ ]:
with open("../src/pipelines/training/payloads/dev.json") as f:
    payload = json.load(f)
    payload = payload["data"]

project_id = os.environ.get("VERTEX_PROJECT_ID")
dataset_id = payload["dataset_id"]
dataset_location = payload["dataset_location"]
data_version = payload["data_version"]

In [ ]:
data_version = get_data_version(
    payload_data_version=data_version,
    project_id=project_id,
    dataset_id=dataset_id,
    dataset_location=dataset_location,
)

In [ ]:
bq_client = bigquery.Client(project=project_id, location=dataset_location)

In [ ]:
test_data = bq_client.query(
    f"SELECT * FROM {project_id}.{dataset_id}_{data_version}.testing"
).to_dataframe()

In [ ]:
test_data = test_data.drop(columns=["transaction_id", "is_fraud"])

In [ ]:
test_data[:10].to_dict("records")